<a href="https://colab.research.google.com/github/hamza-0987/1mg/blob/main/rag_using_groq_web_base_loader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install langchain_groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 18.0 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.13
    Uninstalling langchain-core-0.3.13:
      Successfully uninstalled langchain-core-0.3.13


In [ ]:
pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.4
    Uninstalling langchain-0.3.4:
      Successfully uninstalled langchain-0.3.4


In [ ]:
pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.2 MB/s eta 0:00:00


In [ ]:
import os
from langchain_groq import ChatGroq
from langchain_community.document_loaders import WebBaseLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS
import time

# Load the Groq API key
groq_api_key = "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"

# Load the documents
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()

# Split the documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
final_documents = text_splitter.split_documents(docs[:50])

# Create the embeddings and vector store
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectors = FAISS.from_documents(final_documents, embeddings)

# Create the LLM and prompt
llm = ChatGroq(groq_api_key=groq_api_key, model_name="mixtral-8x7b-32768")
prompt = ChatPromptTemplate.from_template("""
Answer the questions based on the provided context only.
Please provide the most accurate response based on the question
<context>
{context}
<context>
Questions:{input}
""")

# Create the document chain and retrieval chain
document_chain = create_stuff_documents_chain(llm, prompt)
retriever = vectors.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

# Input prompt
prompt_input = input("Enter your prompt: ")

# Process the prompt
start_time = time.process_time()
response = retrieval_chain.invoke({"input": prompt_input})
print("Response time:", time.process_time() - start_time)

# Print the response
print("Answer:", response['answer'])

# Print the relevant chunks
print("Relevant chunks:")
for i, doc in enumerate(response["context"]):
    print(doc.page_content)
    print("--------------------------------")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Enter your prompt: what is langchain
Response time: 0.08936739499999646
Answer: Based on the context provided, LangChain is not explicitly defined. However, it is mentioned that there is no need to use the LangSmith SDK directly if your application is built entirely on LangChain (either Python and JS). This suggests that LangChain is a software development kit or a framework that can be used with LangSmith or on its own. The detailed documentation for LangChain can be found in the links provided in the context.
Relevant chunks:
4. Log your first trace‚Äã
Tracing to LangSmith for LangChain usersThere is no need to use the LangSmith SDK directly if your application is built entirely on LangChain (either Python and JS).We've outlined a tracing guide specifically for LangChain users here.
We provide multiple ways to log traces to LangSmith. Below, we'll highlight
how to use traceable. See more on the Annotate code for tracing page.
--------------------------------
Learn more about evaluati

In [ ]:
import os
import json
from langchain_groq import ChatGroq
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.vectorstores import FAISS
import time

# Load the Groq API key
groq_api_key = "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"

# Load the JSON data from the file
with open('/content/benchmark.json', 'r') as file:
    data = json.load(file)

# Convert JSON data into a list of documents
docs = []
for key, value in data['medqa'].items():
    doc = {
        "title": value['question'],
        "content": value['question'] + ";" + ";".join(value['options'].keys()) + ";" + value['answer']
    }
    docs.append(doc)

# Split the documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
final_documents = text_splitter.split_documents(docs)

# Create the embeddings and vector store
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectors = FAISS.from_documents(final_documents, embeddings)

# Create the LLM and prompt
llm = ChatGroq(groq_api_key=groq_api_key, model_name="mixtral-8x7b-32768")
prompt = ChatPromptTemplate.from_template("""
Answer the questions based on the provided context only.
Please provide the most accurate response based on the question.
<context>
{context}
<context>
Questions: {input}
""")

# Create the document chain and retrieval chain
document_chain = create_stuff_documents_chain(llm, prompt)
retriever = vectors.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

# Input prompt
prompt_input = input("Enter your prompt: ")

# Process the prompt
start_time = time.process_time()
response = retrieval_chain.invoke({"input": prompt_input})
print("Response time:", time.process_time() - start_time)

# Print the response
print("Answer:", response['answer'])

# Print the relevant chunks
print("Relevant chunks:")
for i, doc in enumerate(response["context"]):
    if isinstance(doc, dict) and 'content' in doc:
        print(doc['content'])
    elif isinstance(doc, str):
        print(doc)
    print("--------------------------------")


AttributeError: 'dict' object has no attribute 'page_content'

In [ ]:
import os
import json
from langchain_groq import ChatGroq
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.vectorstores import FAISS
import time

# Load the Groq API key
groq_api_key = "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"

# Load the JSON data from the file
with open('/content/benchmark.json', 'r') as file:
    data = json.load(file)

# Convert JSON data into a list of documents
docs = []
for key, value in data['medqa'].items():
    doc = {
        "title": value['question'],
        "content": value['question'] + ";" + ";".join(value['options'].keys()) + ";" + value['answer'],
        "correct_answer": value['answer']
    }
    docs.append(doc)

# Split the documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
final_documents = text_splitter.split_documents(docs)

# Create the embeddings and vector store
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectors = FAISS.from_documents(final_documents, embeddings)

# Create the LLM and prompt
llm = ChatGroq(groq_api_key=groq_api_key, model_name="mixtral-8x7b-32768")
prompt = ChatPromptTemplate.from_template("""
Answer the questions based on the provided context only.
Please provide the most accurate response based on the question.
<context>
{context}
<context>
Questions: {input}
""")

# Create the document chain and retrieval chain
document_chain = create_stuff_documents_chain(llm, prompt)
retriever = vectors.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

# Input prompt
prompt_input = input("Enter your prompt: ")

# Process the prompt
start_time = time.process_time()
response = retrieval_chain.invoke({"input": prompt_input})
print("Response time:", time.process_time() - start_time)

# Print the model's response
model_answer = response['answer'].strip()
print("Model Answer:", model_answer)

# Retrieve the correct answer from the relevant document
relevant_doc = None
for doc in final_documents:
    if prompt_input.strip().lower() in doc['content'].strip().lower():
        relevant_doc = doc
        break

# Compare the model's answer with the correct answer
if relevant_doc:
    correct_answer = relevant_doc['correct_answer']
    print("Correct Answer:", correct_answer)

    if model_answer == correct_answer:
        print("The model's answer is correct!")
    else:
        print("The model's answer is incorrect.")
else:
    print("Could not find the relevant document for comparison.")

# Print the relevant chunks
print("Relevant chunks:")
for i, doc in enumerate(response["context"]):
    if isinstance(doc, dict) and 'content' in doc:
        print(doc['content'])
    elif isinstance(doc, str):
        print(doc)
    print("--------------------------------")


AttributeError: 'dict' object has no attribute 'page_content'

In [ ]:
import os
import json
from langchain_groq import ChatGroq
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.vectorstores import FAISS
from langchain.schema import Document # Import the Document class
import time

# Load the Groq API key
groq_api_key = "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"

# Load the JSON data from the file
with open('/content/benchmark.json', 'r') as file:
    data = json.load(file)

# Convert JSON data into a list of Document objects
docs = []
for key, value in data['medqa'].items():
    doc = Document(
        page_content=value['question'] + ";" + ";".join(value['options'].keys()) + ";" + value['answer'],
        metadata={"title": value['question'], "correct_answer": value['answer']}
    )
    docs.append(doc)


In [ ]:
import os
import json
from langchain_groq import ChatGroq
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.vectorstores import FAISS
import time
from langchain.schema import Document

# Load the Groq API key
groq_api_key = "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"

# Load the JSON data from the file
with open('/content/benchmark.json', 'r') as file:
    data = json.load(file)

# Convert JSON data into a list of Document objects
docs = []
for key, value in data['medqa'].items():
    doc_content = value['question'] + ";" + ";".join(value['options'].keys()) + ";" + value['answer']
    docs.append(Document(page_content=doc_content, metadata={"correct_answer": value['answer']}))

# Split the documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
final_documents = text_splitter.split_documents(docs)

# Create the embeddings and vector store
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectors = FAISS.from_documents(final_documents, embeddings)

# Create the LLM and prompt
llm = ChatGroq(groq_api_key=groq_api_key, model_name="mixtral-8x7b-32768")
prompt = ChatPromptTemplate.from_template("""
Answer the questions based on the provided context only.
Please provide the most accurate response based on the question.
<context>
{context}
<context>
Questions: {input}
""")

# Create the document chain and retrieval chain
document_chain = create_stuff_documents_chain(llm, prompt)
retriever = vectors.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

# Input prompt
prompt_input = input("Enter your prompt: ")

# Process the prompt
start_time = time.process_time()
response = retrieval_chain.invoke({"input": prompt_input})
print("Response time:", time.process_time() - start_time)

# Print the model's response
model_answer = response['answer'].strip()
print("Model Answer:", model_answer)

# Retrieve the correct answer from the relevant document
relevant_doc = None
for doc in final_documents:
    if prompt_input.strip().lower() in doc.page_content.strip().lower():
        relevant_doc = doc
        break

# Compare the model's answer with the correct answer
if relevant_doc:
    correct_answer = relevant_doc.metadata['correct_answer']
    print("Correct Answer:", correct_answer)

    if model_answer == correct_answer:
        print("The model's answer is correct!")
    else:
        print("The model's answer is incorrect.")
else:
    print("Could not find the relevant document for comparison.")

# Print the relevant chunks
print("Relevant chunks:")
for i, doc in enumerate(response["context"]):
    if isinstance(doc, dict) and 'content' in doc:
        print(doc['content'])
    elif isinstance(doc, str):
        print(doc)
    print("--------------------------------")


ModuleNotFoundError: No module named 'langchain_groq'

In [3]:
pip install gradio

In [4]:
pip install groq_gradio

In [9]:
import gradio as gr
import groq_gradio
import os
os.environ["GROQ_API_KEY"] = "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"
gr.load(
name='llama-3.2-3b-preview',
src=groq_gradio.registry,
).launch()

/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:229: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2c42f27170bec640f0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
